In [81]:
!pip install scikit_learn
!pip install transformers[torch]
!pip install accelerate>={ACCELERATE_MIN_VERSION}
!pip install pandas torch transformers scikit-learn

^C


DEPRECATION: shuup 3.1.0 has a non-standard dependency specifier toml<1pytz>=2015.4,>=0.10.0. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of shuup or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063

[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


DEPRECATION: shuup 3.1.0 has a non-standard dependency specifier toml<1pytz>=2015.4,>=0.10.0. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of shuup or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063

[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip
DEPRECATION: shuup 3.1.0 has a non-standard dependency specifier toml<1pytz>=2015.4,>=0.10.0. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of shuup or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063

[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


DEPRECATION: shuup 3.1.0 has a non-standard dependency specifier toml<1pytz>=2015.4,>=0.10.0. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of shuup or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063

[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Import Libraries


In [86]:
import pandas as pd
import torch
from torch.utils.data import DataLoader, Dataset, random_split
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments, EvalPrediction, pipeline
import os
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.model_selection import train_test_split



Set Environment Variable

In [ ]:
os.environ["HF_HUB_DISABLE_SYMLINKS_WARNING"] = "1"

Define Compute Metrics Function

In [ ]:
def compute_metrics(pred: EvalPrediction):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)  # Get the predicted class indices

    accuracy = accuracy_score(labels, preds)
    f1 = f1_score(labels, preds, average='weighted')  # or 'macro' based on your preference
    precision = precision_score(labels, preds, average='weighted')
    recall = recall_score(labels, preds, average='weighted')

    return {
        'accuracy': accuracy,
        'f1': f1,
        'precision': precision,
        'recall': recall,
    }

Load the Dataset

In [ ]:
# Load the dataset
df = pd.read_csv('01_pc_train_ib.csv')

# Show the first few rows
print(df.head())


      custom_id                                     text  \
0  pla015439pla                               Өтте күшті   
1  pla083193pla  Мәбазар жок .Оте керемет тамаша керемет   
2  pla113624pla             Кушти , дал тура айтады 👍👍👍👍   
3  pla029825pla                              Реклама коп   
4  pla002604pla                           5-баға беремін   

                             text_cleaned  label    domain  
0                              өтте күшті      1  appstore  
1  мәбазар жок оте керемет тамаша керемет      1  appstore  
2                   кушти дал тура айтады      1  appstore  
3                             реклама коп      0  appstore  
4                          5 баға беремін      1  appstore  


Load BERT Tokenizer and Model



In [ ]:
# Load the multilingual BERT tokenizer and model
model_name = 'bert-base-multilingual-cased'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=3)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Create Custom Dataset Class



In [ ]:
class KazakhSentimentDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len=128):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]

        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            truncation=True,
            padding='max_length',
            return_attention_mask=True,
            return_tensors='pt'
        )

        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }


Prepare the Dataset



In [ ]:
texts = df['text_cleaned'].tolist() 
labels = df['label'].tolist()

# Create dataset and split it into training and validation sets
dataset = KazakhSentimentDataset(texts, labels, tokenizer)
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])


Define Training Arguments

In [ ]:
training_args = TrainingArguments(
    output_dir='C:/Users/Toktarla/OneDrive/Desktop/sentiment-analysis-kz/results',
    num_train_epochs=3,
    per_device_train_batch_size=16,  
    per_device_eval_batch_size=16,  
    gradient_accumulation_steps=1,  
    logging_dir='C:/Users/Toktarla/OneDrive/Desktop/sentiment-analysis-kz/logs',
    logging_steps=50,  
    warmup_steps=300,  
    weight_decay=0.01,
    save_strategy="epoch",  
    save_total_limit=2,  
    eval_strategy="epoch", 
    learning_rate=3e-5, 
    fp16=torch.cuda.is_available(), 
)


Create Trainer and Train the Model

In [ ]:
model.gradient_checkpointing_enable()

# Create a Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)

# Train the model
trainer.train()


  0%|          | 0/744 [00:00<?, ?it/s]

c:\Users\Toktarla\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\amp\autocast_mode.py:198: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


{'loss': 0.9253, 'grad_norm': 3.558798313140869, 'learning_rate': 4.9999999999999996e-06, 'epoch': 0.2}
{'loss': 0.4872, 'grad_norm': 2.562490701675415, 'learning_rate': 9.999999999999999e-06, 'epoch': 0.4}
{'loss': 0.4623, 'grad_norm': 4.197021961212158, 'learning_rate': 1.5e-05, 'epoch': 0.6}
{'loss': 0.4078, 'grad_norm': 4.171961307525635, 'learning_rate': 1.9999999999999998e-05, 'epoch': 0.81}


  0%|          | 0/62 [00:00<?, ?it/s]

{'eval_loss': 0.36107271909713745, 'eval_accuracy': 0.8568548387096774, 'eval_f1': 0.8411748917968487, 'eval_precision': 0.8424289413051333, 'eval_recall': 0.8568548387096774, 'eval_runtime': 241.1936, 'eval_samples_per_second': 4.113, 'eval_steps_per_second': 0.257, 'epoch': 1.0}


...

c:\Users\Toktarla\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\amp\autocast_mode.py:198: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


{'loss': 0.2785, 'grad_norm': 5.691867351531982, 'learning_rate': 1.648648648648649e-05, 'epoch': 2.02}
{'loss': 0.2412, 'grad_norm': 5.426058292388916, 'learning_rate': 1.3108108108108109e-05, 'epoch': 2.22}
{'loss': 0.2792, 'grad_norm': 1.2105828523635864, 'learning_rate': 9.72972972972973e-06, 'epoch': 2.42}
{'loss': 0.2511, 'grad_norm': 7.025335788726807, 'learning_rate': 6.351351351351352e-06, 'epoch': 2.62}
{'loss': 0.2119, 'grad_norm': 6.159411430358887, 'learning_rate': 2.972972972972973e-06, 'epoch': 2.82}


  0%|          | 0/62 [00:00<?, ?it/s]

{'eval_loss': 0.4542233943939209, 'eval_accuracy': 0.8518145161290323, 'eval_f1': 0.8424642597138201, 'eval_precision': 0.8392516024279031, 'eval_recall': 0.8518145161290323, 'eval_runtime': 217.429, 'eval_samples_per_second': 4.562, 'eval_steps_per_second': 0.285, 'epoch': 3.0}
{'train_runtime': 13639.5844, 'train_samples_per_second': 0.873, 'train_steps_per_second': 0.055, 'train_loss': 0.36684104447723714, 'epoch': 3.0}


TrainOutput(global_step=744, training_loss=0.36684104447723714, metrics={'train_runtime': 13639.5844, 'train_samples_per_second': 0.873, 'train_steps_per_second': 0.055, 'total_flos': 783025531158528.0, 'train_loss': 0.36684104447723714, 'epoch': 3.0})

Evaluate the Fine-tuned Model

In [85]:
# Evaluate the model on the validation set
eval_result = trainer.evaluate()

# Access the evaluation results
accuracy = eval_result.get('eval_accuracy', eval_result.get('accuracy'))
f1 = eval_result.get('eval_f1', eval_result.get('f1'))
precision = eval_result.get('eval_precision', eval_result.get('precision'))
recall = eval_result.get('eval_recall', eval_result.get('recall'))

print(eval_result)
print(f"Validation Accuracy: {accuracy}")
print(f"Validation F1 Score: {f1}")
print(f"Validation Precision: {precision}")
print(f"Validation Recall: {recall}")


{'eval_loss': 0.4542233943939209, 'eval_accuracy': 0.8518145161290323, 'eval_f1': 0.8424642597138201, 'eval_precision': 0.8392516024279031, 'eval_recall': 0.8518145161290323, 'eval_runtime': 274.0429, 'eval_samples_per_second': 3.62, 'eval_steps_per_second': 0.226, 'epoch': 3.0}
Validation Accuracy: 0.8518145161290323
Validation F1 Score: 0.8424642597138201
Validation Precision: 0.8392516024279031
Validation Recall: 0.8518145161290323


Zero Shot Learning


In [97]:
zero_shot_classifier = pipeline("zero-shot-classification", model="bert-base-multilingual-cased")

candidate_labels = ["positive", "negative"]

def zero_shot_predict(text):
    result = zero_shot_classifier(text, candidate_labels)
    highest_label = result['labels'][0]
    highest_score = result['scores'][0]
    return highest_label, highest_score

def evaluate_zero_shot_model(df):
    true_labels = df['label'].tolist()  # Assuming the true labels are in the 'label' column
    predictions = []
    prediction_scores = []

    for text in df['text_cleaned'].tolist():  # Assuming 'text_cleaned' contains the texts
        prediction, score = zero_shot_predict(text)
        predicted_label = 0 if prediction == "negative" else 1
        predictions.append(predicted_label)
        prediction_scores.append(score)

    accuracy = accuracy_score(true_labels, predictions)
    f1 = f1_score(true_labels, predictions, average='weighted')
    precision = precision_score(true_labels, predictions, average='weighted')
    recall = recall_score(true_labels, predictions, average='weighted')

    print(f"Accuracy: {accuracy}")
    print(f"F1 Score: {f1}")
    print(f"Precision: {precision}")
    print(f"Recall: {recall}")

    print("\nExample Predictions with Confidence Scores:")
    for i in range(5):  # Display first 5 predictions
        print(f"Text: {df['text_cleaned'].iloc[i]}")
        print(f"Prediction: {predictions[i]} ({'positive' if predictions[i] == 1 else 'negative'}), Confidence: {prediction_scores[i]:.4f}")
        print("-" * 50)

df_sample = df.sample(n=100, random_state=42)  # random_state ensures reproducibility

evaluate_zero_shot_model(df_sample)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Failed to determine 'entailment' label id from the label2id mapping in the model config. Setting to -1. Define a descriptive label2id mapping in the model config to ensure correct outputs.


Accuracy: 0.54
F1 Score: 0.6088565488565488
Precision: 0.7184719864176572
Recall: 0.54

Example Predictions with Confidence Scores:
Text: көп орын алмайды дизайны жақсы жеңіл жақсы суытады екі жақ жаны ыстық болып тұрады үнемі
Prediction: 1 (positive), Confidence: 0.5024
--------------------------------------------------
Text: өте керемет фонды істеу
Prediction: 0 (negative), Confidence: 0.5004
--------------------------------------------------
Text: бұл праграмма өте өте қатты ұнады
Prediction: 1 (positive), Confidence: 0.5007
--------------------------------------------------
Text: тамак заказ бердык бырак сан сапасы жок соны айтсак шешып бере алмады
Prediction: 0 (negative), Confidence: 0.5003
--------------------------------------------------
Text: бетті қалай ауыстырады
Prediction: 1 (positive), Confidence: 0.5043
--------------------------------------------------


Few Shot Learning

In [ ]:
model_name = 'bert-base-multilingual-cased'
tokenizer = BertTokenizer.from_pretrained(model_name)
model_few_shot = BertForSequenceClassification.from_pretrained(model_name, num_labels=2)

few_shot_train_size = 100  

# Select a few-shot subset of the dataset
few_shot_df = df.sample(n=few_shot_train_size, random_state=42)

# Prepare the dataset (same as before)
texts_few_shot = few_shot_df['text_cleaned'].tolist()
labels_few_shot = few_shot_df['label'].tolist()

# Create a smaller dataset
train_dataset_few_shot = KazakhSentimentDataset(texts_few_shot, labels_few_shot, tokenizer)

# Split the few-shot data into train and validation sets
train_size_few_shot = int(0.8 * len(train_dataset_few_shot))
val_size_few_shot = len(train_dataset_few_shot) - train_size_few_shot
train_dataset_few_shot, val_dataset_few_shot = random_split(train_dataset_few_shot, [train_size_few_shot, val_size_few_shot])

# Update training arguments for a faster few-shot training process
few_shot_training_args = TrainingArguments(
    output_dir='C:/Users/Toktarla/OneDrive/Desktop/sentiment-analysis-kz/results_few_shot',
    num_train_epochs=3,  # Train with few epochs for faster learning
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    gradient_accumulation_steps=1,
    logging_dir='C:/Users/Toktarla/OneDrive/Desktop/sentiment-analysis-kz/logs_few_shot',
    warmup_steps=50,  # Adjust warmup for faster convergence
    weight_decay=0.01,
    save_strategy="epoch",
    eval_strategy="epoch",
    learning_rate=3e-5,
    fp16=torch.cuda.is_available(),
)

# Create a Trainer for few-shot learning
trainer_few_shot = Trainer(
    model=model_few_shot,
    args=few_shot_training_args,
    train_dataset=train_dataset_few_shot,
    eval_dataset=val_dataset_few_shot,
    compute_metrics=compute_metrics
)

# Fine-tune the model on the few-shot dataset
trainer_few_shot.train()


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

c:\Users\Toktarla\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


{'eval_loss': 0.5710974335670471, 'eval_accuracy': 0.85, 'eval_f1': 0.7810810810810811, 'eval_precision': 0.7224999999999999, 'eval_recall': 0.85, 'eval_runtime': 5.2506, 'eval_samples_per_second': 3.809, 'eval_steps_per_second': 0.381, 'epoch': 1.0}


...

c:\Users\Toktarla\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


{'eval_loss': 0.4457380771636963, 'eval_accuracy': 0.85, 'eval_f1': 0.7810810810810811, 'eval_precision': 0.7224999999999999, 'eval_recall': 0.85, 'eval_runtime': 5.9148, 'eval_samples_per_second': 3.381, 'eval_steps_per_second': 0.338, 'epoch': 3.0}
{'train_runtime': 243.6033, 'train_samples_per_second': 0.985, 'train_steps_per_second': 0.062, 'train_loss': 0.5294204076131185, 'epoch': 3.0}


  0%|          | 0/2 [00:00<?, ?it/s]

{'eval_loss': 0.4457380771636963, 'eval_accuracy': 0.85, 'eval_f1': 0.7810810810810811, 'eval_precision': 0.7224999999999999, 'eval_recall': 0.85, 'eval_runtime': 5.2951, 'eval_samples_per_second': 3.777, 'eval_steps_per_second': 0.378, 'epoch': 3.0}


c:\Users\Toktarla\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
eval_result_few_shot = trainer_few_shot.evaluate()
print(eval_result_few_shot)

  0%|          | 0/2 [00:00<?, ?it/s]

{'eval_loss': 0.4457380771636963, 'eval_accuracy': 0.85, 'eval_f1': 0.7810810810810811, 'eval_precision': 0.7224999999999999, 'eval_recall': 0.85, 'eval_runtime': 5.5298, 'eval_samples_per_second': 3.617, 'eval_steps_per_second': 0.362, 'epoch': 3.0}


c:\Users\Toktarla\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
def predict_sentiment(text):
    encoding = tokenizer.encode_plus(
        text,
        add_special_tokens=True,
        max_length=128,
        truncation=True,
        padding='max_length',
        return_attention_mask=True,
        return_tensors='pt'
    )

    input_ids = encoding['input_ids']
    attention_mask = encoding['attention_mask']

    output = model(input_ids, attention_mask=attention_mask)
    logits = output.logits

    prediction = torch.argmax(logits, dim=1).item()

    return prediction  


Test Prediction with Sample Text


In [ ]:
sample_text = "Я ниче не говорю про суши."
print(f"Predicted Sentiment: {predict_sentiment(sample_text)}")


Predicted Sentiment: 0
